# 1.匯入相關庫，通過網路下載並接收手寫mnist數字庫

In [4]:
import numpy as np
import tensorflow as tf
# 下載並載入MNIST手寫數字庫(55000 * 28 * 28)55000張訓練影象
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [5]:
mnist = input_data.read_data_sets('Mnist_Data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Mnist_Data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Mnist_Data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting Mnist_Data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting Mnist_Data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

one_hot是一種編碼形式，獨熱碼的編碼（encoding）形式 

0， 1， 2， 3， 4， 5， 6， 7， 8， 9的十位數字

one_hot=True 就會表示成下面的編碼模式：

# 0:1000000000
# 1:0100000000
# 2:0010000000
# 3:0001000000
# 4:0000100000
# 5:0000010000
# 6:0000001000
# 7:0000000100
# 8:0000000010
# 9:0000000001

# 2.設定卷積神經網路輸入張量及測試資料 

In [7]:
# None 表示張量（Tensor）的第一個維度可以是任何長度
input_x = tf.placeholder(tf.float32, [None, 28*28]) / 255.
output_y = tf.placeholder(tf.int32, [None, 10]) # 輸出：10個數字的標籤
input_x_images = tf.reshape(input_x, [-1, 28, 28, 1]) # 改變形狀之後的輸入

# Test（測試）資料集裡選取3000個手寫數字的圖片和對應標籤
test_x = mnist.test.images[:3000] # 圖片
test_y = mnist.test.labels[:3000] # 標籤

# None 表示張量（Tensor）的第一個維度可以是任何長度。
# Test（測試）資料集裡選取3000個手寫數字的圖片和對應標籤。

# 3.分層構建卷積神經網路

In [8]:
# 第1層卷積 
conv1 = tf.layers.conv2d(
	inputs=input_x_images,	# 形狀 [28， 28， 1]
	filters=32,		# 32個過濾器， 輸出的深度（depth）是32
	kernel_size=[5, 5],	# 過濾器在二維的大小是(5 * 5)
	strides=1,		# 步長是1
	padding='same',		# same表示輸出的大小不變，因此需要在外圍補零2圈
	activation=tf.nn.relu	# 啟用函式是Relu
)	# 形狀 [28, 28, 32]

# 第1層池化（亞取樣）
pool1 = tf.layers.max_pooling2d(
	inputs=conv1,			# 形狀 [28, 28, 32]
	pool_size=[2, 2],		# 過濾器在二維的大小是(2 * 2)		
	strides=2			# 步長是2
)	# 形狀[14, 14, 32]

# 第2層卷積
conv2 = tf.layers.conv2d(
	inputs=pool1,			# 形狀 [14, 14, 32]
	filters=64,			# 64個過濾器， 輸出的深度（depth）是64
	kernel_size=[5, 5],		# 過濾器在二維的大小是(5 * 5)
	strides=1,			# 步長是1
	padding='same',			# same表示輸出的大小不變，因此需要在外圍補零2圈
	activation=tf.nn.relu		# 啟用函式是Relu
)	# 形狀 [14, 14, 64]

# 第2層池化（亞取樣）
pool2 = tf.layers.max_pooling2d(
	inputs=conv2,			# 形狀 [14, 14, 64]
	pool_size=[2, 2]	,	# 過濾器在二維的大小是(2 * 2)		
	strides=2			# 步長是2
)	# 形狀[7, 7, 64]

# 平坦化（flat）
flat = tf.reshape(pool2, [-1, 7 * 7 *64])	# 形狀 [7 * 7 * 64]

# 1024 個神經元的全連線層
dense = tf.layers.dense(inputs=flat, units=1024, activation=tf.nn.relu)

# Dropout：丟棄50%，rate=0.5
dropout = tf.layers.dropout(inputs=dense, rate=0.5)

# 10個神經元的全連線層，這裡不用啟用函式做非線性化
logits = tf.layers.dense(inputs=dropout, units=10)	# 輸出形狀[1, 1, 10]

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.


# 4.最小化誤差及計算精度

In [10]:
# 計算誤差（計算Cross entropy（交叉熵），再用Softmax計算出百分比概率）
loss = tf.losses.softmax_cross_entropy(onehot_labels=output_y, logits=logits)

# 用Adam優化器來最小化誤差，學習率 0.001
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

# 計算預測值和實際標籤的匹配程度（精度）
# 返回（accuracy, update_op）,會建立兩個區域性變數
accuracy = tf.metrics.accuracy(
	labels=tf.argmax(output_y, axis=1),
	predictions=tf.argmax(logits, axis=1),)[1]

# 我們使用的是adam優化器進行誤差的最小化，此處我們的學習率為0.001 

# 5.訓練神經網路並測試

In [11]:
# 建立會話
sess = tf.Session()

# 初始化變數：
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)

# 訓練神經網路
for i in range(20000):
	batch = mnist.train.next_batch(50) #從Train（訓練)資料集裡取下一個50樣本
	train_loss, _ = sess.run([loss, train_op], {input_x: batch[0], output_y: batch[1]})
	if i % 100 == 0:
		test_accuracy = sess.run(accuracy, {input_x: test_x, output_y: test_y})
		print("Step=%d, Train loss=%.4f, [Test accuracy=%.2f]" % (i, train_loss, test_accuracy))

# 測試：列印20個測試值和真實值
test_output = sess.run(logits, {input_x: test_x[:20]})
inferenced_y = np.argmax(test_output, 1)
print(inferenced_y, 'Inferenced numbers') # 推測的數字
print(np.argmax(test_y[:20], 1), 'Real numbers') # 真實的數字

# 關閉會話
sess.close()

Step=0, Train loss=2.2873, [Test accuracy=0.26]
Step=100, Train loss=0.0563, [Test accuracy=0.59]
Step=200, Train loss=0.0642, [Test accuracy=0.71]
Step=300, Train loss=0.0449, [Test accuracy=0.78]
Step=400, Train loss=0.0468, [Test accuracy=0.81]
Step=500, Train loss=0.0271, [Test accuracy=0.84]
Step=600, Train loss=0.0133, [Test accuracy=0.86]
Step=700, Train loss=0.0585, [Test accuracy=0.88]
Step=800, Train loss=0.0406, [Test accuracy=0.89]
Step=900, Train loss=0.0312, [Test accuracy=0.90]
Step=1000, Train loss=0.0590, [Test accuracy=0.90]
Step=1100, Train loss=0.0404, [Test accuracy=0.91]
Step=1200, Train loss=0.0942, [Test accuracy=0.92]
Step=1300, Train loss=0.0602, [Test accuracy=0.92]
Step=1400, Train loss=0.0775, [Test accuracy=0.92]
Step=1500, Train loss=0.0046, [Test accuracy=0.93]
Step=1600, Train loss=0.0198, [Test accuracy=0.93]
Step=1700, Train loss=0.0152, [Test accuracy=0.93]
Step=1800, Train loss=0.0237, [Test accuracy=0.94]
Step=1900, Train loss=0.0918, [Test accurac

KeyboardInterrupt: 

# 測試：輸出20個測試值和真實值

In [12]:
test_output = sess.run(logits, {input_x: test_x[:20]})
inferenced_y = np.argmax(test_output, 1)
print(inferenced_y, 'Inferenced numbers') # 推測的數字
print(np.argmax(test_y[:20], 1), 'Real numbers') # 真實的數字

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4] Inferenced numbers
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4] Real numbers
